<a href="https://colab.research.google.com/github/SidharthMN/CDLAB/blob/main/roboflow_data_on_remote_train_with%20preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###install drivers

In [1]:
!nvidia-smi

Wed Feb 18 18:08:06 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

###install ultralytics

In [3]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 72.6 MB/s eta 0:00:00


###Install roboflow data

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="pSHla59oUMBHPukNOML1")
project = rf.workspace("sidsoid").project("red-light-violation-xsiyy-pz7co")
version = project.version(2)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 147.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.13.0.92
    Uninstalling opencv-python-headless-4.13.0.92:
      Successfully uninstalled opencv-python-headless-4.13.0.92
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Red-light-violation-2 in yolov8:: 100%|██████████| 23516/23516 [00:04<00:00, 5497.28it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#Preprocessing Steps

##Remove null images

In [5]:
import os
import cv2

# 🔁 CHANGE THIS PATH to your dataset images folder
DATASET_PATH = "/content/dataset/images"  # e.g. /content/train/images

removed = 0
valid = 0

for root, dirs, files in os.walk(DATASET_PATH):
    for file in files:
        file_path = os.path.join(root, file)

        # Check if file is an image (jpg, png, jpeg)
        if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
            print(f"Removing non-image file: {file}")
            os.remove(file_path)
            removed += 1
            continue

        # Check file size (0 KB images)
        if os.path.getsize(file_path) == 0:
            print(f"Removing empty image: {file}")
            os.remove(file_path)
            removed += 1
            continue

        # Try reading the image
        img = cv2.imread(file_path)

        # If image is None (corrupt/null)
        if img is None:
            print(f"Removing corrupt image: {file}")
            os.remove(file_path)
            removed += 1
            continue

        # Check invalid dimensions
        h, w = img.shape[:2]
        if h == 0 or w == 0:
            print(f"Removing invalid dimension image: {file}")
            os.remove(file_path)
            removed += 1
        else:
            valid += 1

print("\n✅ Preprocessing Complete!")
print(f"Valid Images: {valid}")
print(f"Removed (Null/Corrupt) Images: {removed}")


✅ Preprocessing Complete!
Valid Images: 0
Removed (Null/Corrupt) Images: 0


##Brightness

In [6]:
import cv2
import os

# 🔁 CHANGE THESE PATHS
input_folder = os.path.join(dataset.location, "train", "images") # Changed to use dataset.location
output_folder = "/content/dataset_bright/images"

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Brightness factor
# >1.0 = brighter
# <1.0 = darker
brightness_factor = 1.3  # try 1.2 to 1.5 for low-light datasets

processed = 0

for file in os.listdir(input_folder):
    input_path = os.path.join(input_folder, file)

    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
        img = cv2.imread(input_path)

        if img is None:
            continue

        # Convert to float for brightness scaling
        bright_img = cv2.convertScaleAbs(img, alpha=brightness_factor, beta=0)

        # Save image
        output_path = os.path.join(output_folder, file)
        cv2.imwrite(output_path, bright_img)
        processed += 1

print(f"✅ Brightness preprocessing complete! Processed images: {processed}")

✅ Brightness preprocessing complete! Processed images: 10239


##Motion Blur

In [7]:
import cv2
import numpy as np
import os

# 🔁 CHANGE PATHS
input_folder = os.path.join(dataset.location, 'train', 'images') # Corrected path
output_folder = "/content/dataset_motion_blur/images"
os.makedirs(output_folder, exist_ok=True)

# Motion blur strength (0.2 = light blur)
blur_strength = 0.2

def apply_motion_blur(image, strength=0.2):
    h, w = image.shape[:2]

    # Kernel size based on strength (adaptive)
    ksize = max(3, int(min(h, w) * strength * 0.05))

    # Ensure kernel size is odd
    if ksize % 2 == 0:
        ksize += 1

    # Create horizontal motion blur kernel
    kernel = np.zeros((ksize, ksize))
    kernel[int((ksize - 1) / 2), :] = np.ones(ksize)
    kernel = kernel / ksize

    # Apply filter
    blurred = cv2.filter2D(image, -1, kernel)
    return blurred

processed = 0

for file in os.listdir(input_folder):
    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
        img_path = os.path.join(input_folder, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        # Apply motion blur (0.2)
        motion_blurred = apply_motion_blur(img, blur_strength)

        # Save output
        output_path = os.path.join(output_folder, file)
        cv2.imwrite(output_path, motion_blurred)
        processed += 1

print(f"🎯 Motion Blur (0.2) applied to {processed} images!")

🎯 Motion Blur (0.2) applied to 10239 images!


##Blur

In [8]:
import cv2
import os

# ❌ Change paths
input_folder = os.path.join(dataset.location, 'train', 'images') # Corrected path
output_folder = "/content/dataset_blur_1px/images"
os.makedirs(output_folder, exist_ok=True)

processed = 0

for file in os.listdir(input_folder):
    if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
        img_path = os.path.join(input_folder, file)
        img = cv2.imread(img_path)

        if img is None:
            continue

        # 1px blur (very light blur)
        blurred = cv2.GaussianBlur(img, (3, 3), 0)

        # Save image
        cv2.imwrite(os.path.join(output_folder, file), blurred)
        processed += 1

print(f"✅ 1px Blur applied to {processed} images!")

✅ 1px Blur applied to 10239 images!


###Train YOLO (takes time)

In [ ]:
!yolo task=detect mode=train \
  model=yolov8n.pt \
  data={dataset.location}/data.yaml \
  epochs=30 \
  imgsz=512 \
  batch=8 \
  device=0 \
  cache=True


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Bike-Helmet-Detection-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

###Test YOLO Model

Now that you have trained your YOLO model, you can use it to make predictions. The following code will load your trained model and run inference on an image. The results will be saved in the `runs/detect/predict` directory.

In [ ]:
from ultralytics import YOLO

# Load the custom trained model
model = YOLO('runs/detect/train/weights/best.pt')  # Ensure this path is correct

# Run inference on an image
# You can replace 'https://----' with your own image path or a URL
results = model.predict(source='https://static.toiimg.com/thumb/msid-94412660,width-1070,height-580,imgsize-427383,resizemode-75,overlay-toi_sw,pt-32,y_pad-40/photo.jpg', save=True, conf=0.25)

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP came

# download model


In [ ]:
import os
if os.path.exists("runs/detect/train/weights/best.pt"):
  os.rename("runs/detect/train/weights/best.pt", "runs/detect/train/weights/helmet.pt")
  print("Renamed best.pt to helmet.pt")
else:
  print("best.pt not found. Please ensure the training completed successfully.")

from google.colab import files
files.download("runs/detect/train/weights/helmet.pt")

Renamed best.pt to helmet.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.
WARNING ⚠️ Video stream unresponsive, please check your IP camera connection.


# Task
Create a combined dataset for YOLO training by organizing original and augmented image and label files into new training, validation, and test directories, generating a new `data.yaml` file to reflect these changes, and then training a YOLO model using this consolidated dataset. Finally, summarize the data combining process and the outcomes of the YOLO model training.

## Create Combined Dataset Directories




In [9]:
import os

# Define the root directory for the combined dataset
combined_dataset_path = "/content/combined_dataset"

# Create the main combined dataset directory
os.makedirs(combined_dataset_path, exist_ok=True)
print(f"Created directory: {combined_dataset_path}")

# Define the splits and subfolders
splits = ["train", "valid", "test"]
subfolders = ["images", "labels"]

# Create subdirectories for each split (train, valid, test) and their respective images/labels folders
for split in splits:
    for subfolder in subfolders:
        path = os.path.join(combined_dataset_path, split, subfolder)
        os.makedirs(path, exist_ok=True)
        print(f"Created directory: {path}")

print("\n✅ Combined dataset directory structure created successfully!")

Created directory: /content/combined_dataset
Created directory: /content/combined_dataset/train/images
Created directory: /content/combined_dataset/train/labels
Created directory: /content/combined_dataset/valid/images
Created directory: /content/combined_dataset/valid/labels
Created directory: /content/combined_dataset/test/images
Created directory: /content/combined_dataset/test/labels

✅ Combined dataset directory structure created successfully!


## Copy Original Dataset Files



In [10]:
import shutil
import os

# Define the base paths
original_dataset_base_path = dataset.location # This comes from the roboflow download
combined_dataset_base_path = "/content/combined_dataset"

# Splits to process
splits = ["train", "valid", "test"]

for split in splits:
    # Define source paths for images and labels
    source_images_path = os.path.join(original_dataset_base_path, split, "images")
    source_labels_path = os.path.join(original_dataset_base_path, split, "labels")

    # Define destination paths in the combined dataset
    dest_images_path = os.path.join(combined_dataset_base_path, split, "images")
    dest_labels_path = os.path.join(combined_dataset_base_path, split, "labels")

    print(f"\nCopying original {split} data...")

    # Copy images
    if os.path.exists(source_images_path):
        for file_name in os.listdir(source_images_path):
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
                shutil.copy(os.path.join(source_images_path, file_name), dest_images_path)
        print(f"  Copied {len([f for f in os.listdir(source_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp'))])} images to {dest_images_path}")
    else:
        print(f"  Source images path not found: {source_images_path}")

    # Copy labels
    if os.path.exists(source_labels_path):
        for file_name in os.listdir(source_labels_path):
            if file_name.lower().endswith(('.txt')):
                shutil.copy(os.path.join(source_labels_path, file_name), dest_labels_path)
        print(f"  Copied {len([f for f in os.listdir(source_labels_path) if f.lower().endswith(('.txt'))])} labels to {dest_labels_path}")
    else:
        print(f"  Source labels path not found: {source_labels_path}")

print("\n✅ Original dataset files copied successfully to combined dataset!")


Copying original train data...
  Copied 10239 images to /content/combined_dataset/train/images
  Copied 10239 labels to /content/combined_dataset/train/labels

Copying original valid data...
  Copied 1018 images to /content/combined_dataset/valid/images
  Copied 1018 labels to /content/combined_dataset/valid/labels

Copying original test data...
  Copied 495 images to /content/combined_dataset/test/images
  Copied 495 labels to /content/combined_dataset/test/labels

✅ Original dataset files copied successfully to combined dataset!


## Copy Augmented Brightened Images and Labels


In [11]:
import shutil
import os

# Source paths for augmented images and original labels
brightened_images_source_path = "/content/dataset_bright/images"
original_labels_source_path = os.path.join(dataset.location, "train", "labels")

# Destination paths in the combined dataset (train split)
combined_train_images_dest_path = os.path.join(combined_dataset_base_path, "train", "images")
combined_train_labels_dest_path = os.path.join(combined_dataset_base_path, "train", "labels")

print("\nCopying augmented brightened images and their original labels to combined dataset (train split)...\n")

processed_images = 0
processed_labels = 0

# Copy brightened images and their corresponding labels
if os.path.exists(brightened_images_source_path):
    for image_file in os.listdir(brightened_images_source_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
            source_image_path = os.path.join(brightened_images_source_path, image_file)
            dest_image_path = os.path.join(combined_train_images_dest_path, "bright_" + image_file) # Add prefix to avoid name collision
            shutil.copy(source_image_path, dest_image_path)
            processed_images += 1

            # Construct corresponding label file name
            label_file = os.path.splitext(image_file)[0] + ".txt"
            source_label_path = os.path.join(original_labels_source_path, label_file)
            dest_label_path = os.path.join(combined_train_labels_dest_path, "bright_" + label_file) # Add prefix

            if os.path.exists(source_label_path):
                shutil.copy(source_label_path, dest_label_path)
                processed_labels += 1
            else:
                print(f"Warning: Label file not found for {image_file} at {source_label_path}")

    print(f"  Copied {processed_images} brightened images to {combined_train_images_dest_path}")
    print(f"  Copied {processed_labels} corresponding labels to {combined_train_labels_dest_path}")
else:
    print(f"  Brightened images source path not found: {brightened_images_source_path}")

print("\n✅ Augmented brightened images and labels copied successfully!")


Copying augmented brightened images and their original labels to combined dataset (train split)...

  Copied 10239 brightened images to /content/combined_dataset/train/images
  Copied 10239 corresponding labels to /content/combined_dataset/train/labels

✅ Augmented brightened images and labels copied successfully!


## Copy Augmented Motion-Blurred Images and Labels


In [12]:
import shutil
import os

# Source paths for augmented images and original labels
motion_blurred_images_source_path = "/content/dataset_motion_blur/images"
original_labels_source_path = os.path.join(dataset.location, "train", "labels")

# Destination paths in the combined dataset (train split)
combined_train_images_dest_path = os.path.join(combined_dataset_base_path, "train", "images")
combined_train_labels_dest_path = os.path.join(combined_dataset_base_path, "train", "labels")

print("\nCopying augmented motion-blurred images and their original labels to combined dataset (train split)...\n")

processed_images = 0
processed_labels = 0

# Copy motion-blurred images and their corresponding labels
if os.path.exists(motion_blurred_images_source_path):
    for image_file in os.listdir(motion_blurred_images_source_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
            source_image_path = os.path.join(motion_blurred_images_source_path, image_file)
            dest_image_path = os.path.join(combined_train_images_dest_path, "motionblur_" + image_file) # Add prefix to avoid name collision
            shutil.copy(source_image_path, dest_image_path)
            processed_images += 1

            # Construct corresponding label file name
            label_file = os.path.splitext(image_file)[0] + ".txt"
            source_label_path = os.path.join(original_labels_source_path, label_file)
            dest_label_path = os.path.join(combined_train_labels_dest_path, "motionblur_" + label_file) # Add prefix

            if os.path.exists(source_label_path):
                shutil.copy(source_label_path, dest_label_path)
                processed_labels += 1
            else:
                print(f"Warning: Label file not found for {image_file} at {source_label_path}")

    print(f"  Copied {processed_images} motion-blurred images to {combined_train_images_dest_path}")
    print(f"  Copied {processed_labels} corresponding labels to {combined_train_labels_dest_path}")
else:
    print(f"  Motion-blurred images source path not found: {motion_blurred_images_source_path}")

print("\n✅ Augmented motion-blurred images and labels copied successfully!")


Copying augmented motion-blurred images and their original labels to combined dataset (train split)...

  Copied 10239 motion-blurred images to /content/combined_dataset/train/images
  Copied 10239 corresponding labels to /content/combined_dataset/train/labels

✅ Augmented motion-blurred images and labels copied successfully!


## Copy Augmented 1px Blurred Images and Labels.

In [13]:
import shutil
import os

# Source paths for augmented images and original labels
blurred_1px_images_source_path = "/content/dataset_blur_1px/images"
original_labels_source_path = os.path.join(dataset.location, "train", "labels")

# Destination paths in the combined dataset (train split)
combined_train_images_dest_path = os.path.join(combined_dataset_base_path, "train", "images")
combined_train_labels_dest_path = os.path.join(combined_dataset_base_path, "train", "labels")

print("\nCopying augmented 1px blurred images and their original labels to combined dataset (train split)...\n")

processed_images = 0
processed_labels = 0

# Copy 1px blurred images and their corresponding labels
if os.path.exists(blurred_1px_images_source_path):
    for image_file in os.listdir(blurred_1px_images_source_path):
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.webp')):
            source_image_path = os.path.join(blurred_1px_images_source_path, image_file)
            dest_image_path = os.path.join(combined_train_images_dest_path, "blur1px_" + image_file) # Add prefix to avoid name collision
            shutil.copy(source_image_path, dest_image_path)
            processed_images += 1

            # Construct corresponding label file name
            label_file = os.path.splitext(image_file)[0] + ".txt"
            source_label_path = os.path.join(original_labels_source_path, label_file)
            dest_label_path = os.path.join(combined_train_labels_dest_path, "blur1px_" + label_file) # Add prefix

            if os.path.exists(source_label_path):
                shutil.copy(source_label_path, dest_label_path)
                processed_labels += 1
            else:
                print(f"Warning: Label file not found for {image_file} at {source_label_path}")

    print(f"  Copied {processed_images} 1px blurred images to {combined_train_images_dest_path}")
    print(f"  Copied {processed_labels} corresponding labels to {combined_train_labels_dest_path}")
else:
    print(f"  1px blurred images source path not found: {blurred_1px_images_source_path}")

print("\n✅ Augmented 1px blurred images and labels copied successfully!")


Copying augmented 1px blurred images and their original labels to combined dataset (train split)...

  Copied 10239 1px blurred images to /content/combined_dataset/train/images
  Copied 10239 corresponding labels to /content/combined_dataset/train/labels

✅ Augmented 1px blurred images and labels copied successfully!


## Generate New `data.yaml` File


In [14]:
import yaml
import os

# Define the path for the new data.yaml file
combined_dataset_path = "/content/combined_dataset"
data_yaml_path = os.path.join(combined_dataset_path, "data.yaml")

# Define relative paths for train, valid, and test splits
train_path = os.path.join(combined_dataset_path, "train", "images")
val_path = os.path.join(combined_dataset_path, "valid", "images")
test_path = os.path.join(combined_dataset_path, "test", "images")

# Try to get class names and number of classes from the original dataset's data.yaml
original_data_yaml_path = os.path.join(dataset.location, "data.yaml")

class_names = []
num_classes = 0

if os.path.exists(original_data_yaml_path):
    with open(original_data_yaml_path, 'r') as f:
        original_data = yaml.safe_load(f)
    if 'names' in original_data:
        class_names = original_data['names']
        num_classes = len(class_names)
    else:
        print("Warning: 'names' key not found in original data.yaml. Using default class names.")
else:
    print("Warning: Original data.yaml not found. Using default class names.")

# Fallback for class names if not found
if not class_names:
    # Assuming default classes if not found in original data.yaml or dataset object
    class_names = ['class0', 'class1'] # Placeholder, adjust if specific names are known
    num_classes = len(class_names)

# Construct the content for the new data.yaml
data_content = {
    'path': combined_dataset_path, # Absolute path to the dataset root
    'train': 'train/images',       # Relative path from 'path'
    'val': 'valid/images',         # Relative path from 'path'
    'test': 'test/images',         # Relative path from 'path'
    'nc': num_classes,
    'names': class_names
}

# Write the content to the new data.yaml file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_content, f, sort_keys=False)

print(f"✅ New data.yaml created successfully at: {data_yaml_path}")
print("Content of new data.yaml:")
with open(data_yaml_path, 'r') as f:
    print(f.read())


✅ New data.yaml created successfully at: /content/combined_dataset/data.yaml
Content of new data.yaml:
path: /content/combined_dataset
train: train/images
val: valid/images
test: test/images
nc: 9
names:
- bicycle
- bus
- car
- green_light
- motobike
- red_light
- stop_line
- truck
- yellow_light



## Train YOLO Model with Combined Dataset



In [16]:
import os

# Define the path to the new data.yaml file
combined_dataset_path = "/content/combined_dataset" # Re-defining for this cell's scope
data_yaml_path = os.path.join(combined_dataset_path, "data.yaml")

!yolo task=detect mode=train \
  model=yolov8n.pt \
  data={data_yaml_path} \
  epochs=10 \
  imgsz=416 \
  batch=16 \
  device=0 \
  cache=True

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/combined_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspecti

#Model Evaluation

In [17]:
from ultralytics import YOLO
import os

# Define the path to the combined dataset's data.yaml
combined_dataset_path = "/content/combined_dataset"
data_yaml_path = os.path.join(combined_dataset_path, "data.yaml")

# Load the custom trained model (replace 'train2' with your actual run directory if different)
# The last training run produced results in '/content/runs/detect/train2/weights/best.pt'
model_path = '/content/runs/detect/train2/weights/best.pt'

# Check if the model exists before loading
if os.path.exists(model_path):
    model = YOLO(model_path)
    print(f"✅ Model loaded successfully from: {model_path}")
else:
    print(f"❌ Model not found at: {model_path}. Please ensure training completed successfully and check the path.")
    # You might want to exit or handle this error more gracefully


✅ Model loaded successfully from: /content/runs/detect/train2/weights/best.pt


##Confusion Metics and Accuracy Score

In [18]:
# Run validation on the test dataset to get metrics and confusion matrix
# This will save results, including plots and a confusion matrix, in a new 'predict' directory within the current run folder.

# The 'val' directory might be 'val2' if there were previous validation runs.
metrics = model.val(data=data_yaml_path, split='test', imgsz=416, batch=16, save_json=True, plots=True)

# Print the metrics
print("\n✅ Validation Metrics:")
print(f"mAP50-95: {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")
print(f"mAP75: {metrics.box.map75}")

# The confusion matrix and other plots are saved in the 'runs/detect/valX' directory (e.g., 'runs/detect/val2')
print("\n📄 Confusion matrix and other plots are saved in the run directory (e.g., '/content/runs/detect/val').")
print("Look for files like 'confusion_matrix.png', 'results.png', etc.")


Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
Model summary (fused): 73 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 17.7±8.7 MB/s, size: 48.7 KB)
val: Scanning /content/combined_dataset/test/labels... 495 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 495/495 534.6it/s 0.9s
val: New cache created: /content/combined_dataset/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 31/31 5.8it/s 5.4s
                   all        495       2210      0.768      0.827      0.845      0.622
               bicycle         32         70      0.759      0.729      0.668      0.504
                   bus         53         66      0.815      0.879      0.935      0.822
                   car        223        486      0.806      0.809      0.898      0.713
           green_light        115        199      0.637      0.811      0

In [19]:
import os
from google.colab import files

# Define the path to the newly trained model from the 'train2' run
model_source_path = "runs/detect/train2/weights/best.pt"
model_destination_name = "yolo1v8s.pt"

if os.path.exists(model_source_path):
  # If the file hasn't been renamed already, rename it
  if not os.path.exists(model_destination_name):
    os.rename(model_source_path, model_destination_name)
    print(f"Renamed {model_source_path} to {model_destination_name}")
  else:
    print(f"Model already renamed and available as {model_destination_name}")

  # Download the model
  files.download(model_destination_name)
else:
  print(f"Model not found at: {model_source_path}. Please ensure the training completed successfully.")

Renamed runs/detect/train2/weights/best.pt to yolo1v8s.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>